In [58]:
import numpy
import sympy
from sympy import *
from sympy.matrices import *

In [12]:
def triangle_area_2D(a, b, c):
    return ((b[0] - a[0]) * (c[1] - a[1]) - (c[0] - a[0]) * (b[1] - a[1])) / 2

In [13]:
def tetrahedron_volume(a, b, c, d):
    return numpy.cross(b - a, c - a).dot(d - a) / 6

In [80]:
def cxxcode(expr, assign_to):
    CSE_results = cse(expr, numbered_symbols("t"), optimizations='basic')
    lines = []
    for helper in CSE_results[0]:
        if isinstance(helper[1], MatrixSymbol):
            lines.append(f'const auto {helper[0]}[{helper[1].size}];')
            lines.append(sympy.printing.cxxcode(helper[1], helper[0]))
        else:
            lines.append(
                f'const auto {sympy.printing.cxxcode(helper[0])} = {sympy.printing.cxxcode(helper[1])};')

    for i, result in enumerate(CSE_results[1]):
        lines.append(sympy.printing.cxxcode(result, assign_to))

    return '\n'.join(lines)

# def jac_code(J, var_name):
#     J = sympy.Matrix(J.flatten(order="F"))
#     params = ",".join(
#         [f"double {ccode(var)}" for var in x] 
#         + [f"double {ccode(var)}" for var in ut] 
#         + [f"double {ccode(var)}" for var in u] 
#         + [f"double J[{prod(J.shape)}]"])
#     return(f"""
# void jac_F_wrt_{var_name}({params}){{
# {cxx_print(J)}
# }}""")

In [90]:
def gradient_code(expr_name, expr, x):
    grad = [expr.diff(x_i) for x_i in x]
    params = [f"double {ccode(var)}" for var in x] + [f"double g[{len(grad)}]"]
    return (f"""
void {expr_name}_gradient({",".join(params)}){{
{cxxcode(sympy.Matrix(grad), "g")}
}}""")

def hessian_code(expr_name, expr, x):
    grad = [expr.diff(x_i) for x_i in x]
    hess = sympy.Matrix([[grad_i.diff(x_j) for x_j in x] for grad_i in grad])
    # display(hess)
    params = [f"double {ccode(var)}" for var in x] + [f"double H[{prod(hess.shape)}]"]
    return (f"""
void {expr_name}_hessian({",".join(params)}){{
{cxxcode(hess, "H")}
}}""")

In [91]:
a = numpy.array(sympy.symbols("ax ay az"))
b = numpy.array(sympy.symbols("bx by bz"))
c = numpy.array(sympy.symbols("cx cy cz"))
d = numpy.array(sympy.symbols("dx dy dz"))

In [95]:
f = open("element_volume_derivatives.cpp", "w")

In [96]:
x = numpy.stack([a[:2], b[:2], c[:2]]).flatten()
f.write(gradient_code("triangle_area_2D", triangle_area_2D(a[:2], b[:2], c[:2]), x))
f.write(hessian_code("triangle_area_2D", triangle_area_2D(a[:2], b[:2], c[:2]), x))

571

In [97]:
x = numpy.stack([a, b, c, d]).flatten()
f.write(gradient_code("tetrahedron_volume", tetrahedron_volume(a, b, c, d), x))
f.write(hessian_code("tetrahedron_volume", tetrahedron_volume(a, b, c, d), x))

3649

In [98]:
f.close()